### 대학가 원룸 가격, 이대로 괜찮은가?
기숙사 수용능력이 서울 대학가 원룸 가격에 미치는 영향: 
회귀분석을 통한 연구

### 2. 월세에 대한 회귀분석

In [91]:
import pandas as pd

# Excel 파일 읽어들이기
data = pd.read_excel('./univ_one_room_yonsei_apart_final.xlsx')
# 데이터 출력
print(data)

                   univ_name   item_id  유형   보증금   월세       구_동            주소  \
0                숭실대학교,중앙대학교  37269631  월세   100   23   관악구 봉천동   서울시 관악구 봉천동   
1                숭실대학교,중앙대학교  37269635  월세   100   23   관악구 봉천동   서울시 관악구 봉천동   
2                숭실대학교,중앙대학교  37986523  월세   100   15   관악구 봉천동   서울시 관악구 봉천동   
3                      서울대학교  38175770  월세  2000   35   관악구 봉천동   서울시 관악구 봉천동   
4                      서울대학교  38227769  월세   100   30   관악구 봉천동   서울시 관악구 봉천동   
...                      ...       ...  ..   ...  ...       ...           ...   
5972             명지대학교,상명대학교  38606657  월세  5000   25   은평구 신사동   서울시 은평구 신사동   
5973       건국대학교,세종대학교,한양대학교  38619308  월세   300   35   성동구 송정동   서울시 성동구 송정동   
5974  감리교신학대학교,동국대학교,숙명여자대학교  38621223  월세   500  130  중구 충무로4가  서울시 중구 충무로4가   
5975            국민대학교,성균관대학교  38624328  월세  1000   42   종로구 신교동   서울시 종로구 신교동   
5976            국민대학교,성균관대학교  38624328  월세  1000   42   종로구 신교동   서울시 종로구 신교동   

       관리비   층수  건물_층수  ...

In [76]:
data.isnull().sum()

univ_name                        0
item_id                          0
유형                               0
보증금                              0
월세                               0
구_동                              0
주소                               0
관리비                              0
층수                               0
건물_층수                            0
전용면적                             0
is_new                           0
건물_형태                            0
엘리베이터                            0
룸_형태                             0
집_방향                             0
pnu                              0
냉장고                              0
세탁기                              0
옵션_수                             0
주차여부                             0
지하철역                             0
세탁소                              0
카페                               0
약국                               0
대형마트                             0
편의점                              0
버스정류장                            0
lat                 

In [94]:
data = data[data["min"] <= 3.0]

In [95]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

#층수 뺀 상황
# Selecting X variables
X_vars = ["전용면적","is_new", "건물_형태", "엘리베이터", "룸_형태", "건물_층수", "집_방향", "냉장고","세탁기","옵션_수", 
          "주차여부", "지하철역", "세탁소", "카페", "약국", "대형마트", "편의점", "버스정류장","min", "외국인_학생_수_weighted_mean", 
          "연평균_등록금_weighted_mean", "기숙사비_weighted_mean", "재학생_수_weighted_mean", "수용가능인원_weighted_mean", "지원자_수_weighted_mean", "외국학생할당량_weighted_mean", "기숙사_연령_weighted_mean", "지방_학생_신입생_비율_weighted_mean", "미사용토지_면적_weighted_mean", "전체_교육용토지_면적_weighted_mean", "대학교_학령인구_남자", "대학교_학령인구_여자", "지역학사_개수", "행복기숙사_연합_개수", "기숙사형청년주택", "transaction_real_price_median"]

# Creating dummy variables for categorical columns
X = pd.get_dummies(data[X_vars], drop_first=True)
y = (data["보증금"] / 12 * 0.055) + data["월세"] + data["관리비"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train['is_new'] = X_train['is_new'].astype(int)
X_train['엘리베이터'] = X_train['엘리베이터'].astype(int)
# Regression analysis
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Calculate R2 and RMSE
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Using statsmodels for detailed statistics
X_const = sm.add_constant(X_train)  # adding a constant
ols_model = sm.OLS(y_train, X_const).fit()

r2, rmse, ols_model.summary()

(0.6414524546601966,
 25.292034107848348,
 <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.645
 Model:                            OLS   Adj. R-squared:                  0.641
 Method:                 Least Squares   F-statistic:                     176.0
 Date:                Sat, 11 Nov 2023   Prob (F-statistic):               0.00
 Time:                        00:18:39   Log-Likelihood:                -19485.
 No. Observations:                4214   AIC:                         3.906e+04
 Df Residuals:                    4170   BIC:                         3.934e+04
 Df Model:                          43                                         
 Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
 --------

In [97]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
        
        # print the coefficients and adjusted R-squared after each step
        print("\nCoefficients:")
        print(model.params)
        print("\nAdj. R-squared:")
        print(model.rsquared_adj)

    return included

result = stepwise_selection(X, y)

print('resulting features:', result)

Add  주차여부_주차 불가능                    with p-value 0.0

Coefficients:
const          104.044798
주차여부_주차 불가능    -41.728747
dtype: float64

Adj. R-squared:
0.24700456980601238
Add  전용면적                           with p-value 0.0

Coefficients:
const          46.733554
주차여부_주차 불가능   -17.969620
전용면적            1.519259
dtype: float64

Adj. R-squared:
0.5039321488837343
Add  기숙사비_weighted_mean             with p-value 7.39852e-62

Coefficients:
const                 22.347815
주차여부_주차 불가능          -19.726640
전용면적                   1.508903
기숙사비_weighted_mean     0.936025
dtype: float64

Adj. R-squared:
0.5291345759164343
Add  엘리베이터                          with p-value 4.59585e-66

Coefficients:
const                 12.303538
주차여부_주차 불가능          -13.268239
전용면적                   1.412560
기숙사비_weighted_mean     0.973792
엘리베이터                 15.737193
dtype: float64

Adj. R-squared:
0.554697642591101
Add  transaction_real_price_median  with p-value 5.11393e-72

Coefficients:
const            

Add  외국학생할당량_weighted_mean          with p-value 3.30071e-05

Coefficients:
const                           -55.583458
주차여부_주차 불가능                      -8.178225
전용면적                              1.154461
기숙사비_weighted_mean                1.682447
엘리베이터                            14.323293
transaction_real_price_median     0.000237
룸_형태_오픈형원룸                      -15.765844
옵션_수                              1.762623
재학생_수_weighted_mean              -0.004640
전체_교육용토지_면적_weighted_mean         0.000005
지역학사_개수                          -1.201968
외국인_학생_수_weighted_mean            0.011995
지하철역                             -0.013125
지방_학생_신입생_비율_weighted_mean       86.342240
수용가능인원_weighted_mean              0.014516
세탁소                               0.018391
외국학생할당량_weighted_mean            -0.032035
dtype: float64

Adj. R-squared:
0.6370558683329701
Add  건물_형태_원룸                       with p-value 3.33061e-05

Coefficients:
const                           -50.181443
주차여부_주차 불가능            

In [88]:
#층수 뺀 상황
# Selecting X variables
X_vars = ["전용면적","is_new", "건물_형태", "엘리베이터", "룸_형태","세탁기","옵션_수", 
          "주차여부", "지하철역", "세탁소", "약국", "대형마트", "외국인_학생_수_weighted_mean", 
          "연평균_등록금_weighted_mean", "기숙사비_weighted_mean", "재학생_수_weighted_mean", "수용가능인원_weighted_mean", "지원자_수_weighted_mean", "외국학생할당량_weighted_mean", "지방_학생_신입생_비율_weighted_mean",  "transaction_real_price_median"]
# ['주차여부_주차 불가능', '전용면적', 'transaction_real_price_median', '엘리베이터', '룸_형태_오픈형원룸', '기숙사비_weighted_mean', '재학생_수_weighted_mean', '전체_교육용토지_면적_weighted_mean', '옵션_수', '외국인_학생_수_weighted_mean', '지하철역', '지방_학생_신입생_비율_weighted_mean', '수용가능인원_weighted_mean', '외국학생할당량_weighted_mean', '건물_형태_원룸', '세탁소', '대형마트', '연평균_등록금_weighted_mean', '약국', 'is_new']
# Creating dummy variables for categorical columns
X = pd.get_dummies(data[X_vars], drop_first=True)
y = (data["보증금"] / 12 * 0.055) + data["월세"] + data["관리비"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train['is_new'] = X_train['is_new'].astype(int)
X_train['엘리베이터'] = X_train['엘리베이터'].astype(int)
# Regression analysis
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Calculate R2 and RMSE
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Using statsmodels for detailed statistics
X_const = sm.add_constant(X_train)  # adding a constant
ols_model = sm.OLS(y_train, X_const).fit()

r2, rmse, ols_model.summary()

(0.66063607880896,
 21.09153059441238,
 <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.630
 Model:                            OLS   Adj. R-squared:                  0.628
 Method:                 Least Squares   F-statistic:                     368.3
 Date:                Sat, 11 Nov 2023   Prob (F-statistic):               0.00
 Time:                        00:14:27   Log-Likelihood:                -22198.
 No. Observations:                4781   AIC:                         4.444e+04
 Df Residuals:                    4758   BIC:                         4.459e+04
 Df Model:                          22                                         
 Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
 -----------

### 전세금 변수 추가

In [25]:
# Calculate 전세금 based on the given formula
data['전세금'] = (data['월세'] * 12 * 100) / 5 + data['보증금']

# Display the first few rows of the updated dataframe
data[['월세', '보증금', '전세금']].head()


,월세,보증금,전세금
0,23,100,5620.0
1,23,100,5620.0
2,15,100,3700.0
3,35,2000,10400.0
4,30,100,7300.0


In [26]:
# Setting target variable
y_jeonse = data["전세금"]

# Splitting the data into training and testing sets
X_train_jeonse, X_test_jeonse, y_train_jeonse, y_test_jeonse = train_test_split(X, y_jeonse, test_size=0.2, random_state=42)
X_train_jeonse['is_new'] = X_train_jeonse['is_new'].astype(int)
X_train_jeonse['엘리베이터'] = X_train_jeonse['엘리베이터'].astype(int)
# Regression analysis for 전세금
model_jeonse = LinearRegression()
model_jeonse.fit(X_train_jeonse, y_train_jeonse)

# Predictions
y_pred_jeonse = model_jeonse.predict(X_test_jeonse)

# Calculate R2 and RMSE for 전세금
r2_jeonse = r2_score(y_test_jeonse, y_pred_jeonse)
rmse_jeonse = mean_squared_error(y_test_jeonse, y_pred_jeonse, squared=False)

# Using statsmodels for detailed statistics for 전세금
X_const_jeonse = sm.add_constant(X_train_jeonse)  # adding a constant
ols_model_jeonse = sm.OLS(y_train_jeonse, X_const_jeonse).fit()

r2_jeonse, rmse_jeonse, ols_model_jeonse.summary()


(0.668223399297579,
 5170.0509445355265,
 <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                    전세금   R-squared:                       0.655
 Model:                            OLS   Adj. R-squared:                  0.649
 Method:                 Least Squares   F-statistic:                     112.6
 Date:                Fri, 10 Nov 2023   Prob (F-statistic):               0.00
 Time:                        00:01:43   Log-Likelihood:                -50223.
 No. Observations:                5016   AIC:                         1.006e+05
 Df Residuals:                    4932   BIC:                         1.012e+05
 Df Model:                          83                                         
 Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
 -------------

In [14]:
# Extracting Adjusted R2 values for 월세 and 전세금
adj_r2_wolse = ols_model.rsquared_adj
adj_r2_jeonse = ols_model_jeonse.rsquared_adj

adj_r2_wolse, adj_r2_jeonse

(0.542487846892322, 0.6289639229045267)